# Feedforward Neural Network built on Keras

Below is a program built on Keras to build a regression model. 

## Dependencies

In this work, we need the numpy and pandas for processing raw data, and we will use keras (Tensorflow backend) an scikit-learn for building and evaluating regression models.

In [4]:
import numpy as np
import pandas as pd
import keras.callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


## More preprocessing

In this step, we put all the idle and wild data into one file for the purpose of training and testing a regression model later. We also get rid of timestamps because we are using the emg and x,y,z values (features) at one moment and predicting the x,y,z values (targets) at the next moment. We could add timestamps into features as well.

In [5]:
files = ['miller_idleprocessed', 'sam_wildprocessed', 'layton_idleprocessed', 'miller_wildprocessed', 'val_idleprocessed', 
         'layton_wildprocessed', 'sam_idleprocessed', 'val_wildprocessed']
frames = [pd.read_csv(f, header = None) for f in files]
dataframe = pd.concat(frames)   #Concatenate all preprocessed files into one file

In [6]:
dataset_new = dataframe.values
dataset_new = np.delete(dataset_new, [1, 5], axis = 1)  #delete the second and the sixth items (timestamps) in each array
X = dataset_new[:-1]  # X is the features set
Y = dataset_new[1:]   # Y is the target set
Y = np.delete(Y, [0], axis = 1)  # Y is the targeted set and we don't need the EMG value in Y. Only x, y, z.
print (X,Y)

[[  4.07200000e+03  -3.28000000e-01   2.26500000e-01  -9.49000000e-01]
 [  4.04300000e+03  -3.16000000e-01   2.14800000e-01  -9.37000000e-01]
 [  4.04200000e+03  -3.35000000e-01   2.26500000e-01  -9.45000000e-01]
 ..., 
 [  3.48300000e+03  -5.40000000e-02  -7.07000000e-01   6.25000000e-01]
 [  3.49500000e+03  -4.20000000e-02  -7.14000000e-01   6.17100000e-01]
 [  3.48400000e+03  -5.80000000e-02  -7.26000000e-01   6.17100000e-01]] [[-0.316   0.2148 -0.937 ]
 [-0.335   0.2265 -0.945 ]
 [-0.324   0.2382 -0.937 ]
 ..., 
 [-0.042  -0.714   0.6171]
 [-0.058  -0.726   0.6171]
 [-0.042  -0.746   0.6171]]


## Build the model by using Keras

* In this part, I benefited a lot from this post: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/.
* I put 16 hidden layers and use relu as the activation layer. (It turns out that if I put the number of hidden layers under 10, it doesn't really work.) I use mean_squared_error as the loss function. And I use the adam optimizer. (Adam is better than RMSprop based on my experiment. In one experiment I did with RMSprop, the r2 score is only around 0.89, which is around 9 percent less than when using adam. RMSprop optimizer also learns slower.)
* The official Keras document on Sequential model can be found here: https://keras.io/getting-started/sequential-model-guide/

In [7]:
def baseline_model():
    model = Sequential()
    model.add(Dense(24, input_dim = 4, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(3, kernel_initializer = 'normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

## Testing performance of the model

* A large part of the code here is from Layton. It uses the modules in scikit-learn. The normal scoring of my model, I believe, is mean sqaured error. I changed into r2 in order to have better comparison metrics. It we get rid of  *scoring = 'r2'*, then the printed resulted reg.score will be mean squared error. 
* Here are a couple helpful links from scikit-learn documents on how to set parameters for cross validation here. 
    - http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    - http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [9]:

# from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

from sklearn.model_selection import GridSearchCV

#hyperparameters to search through with CV
parameters = {
    'epochs' : [800, 1600],
    'batch_size' : [12, 36, 64, 128], # The larger the batch size, the faster the model will run.
        'verbose' : [2] # for debugging purpose. It will significantly lower the running speed. To speed up, change it to 0.
}

#searching through all combinations of parameters to find best (ranked by CV)
reg = GridSearchCV(KerasRegressor(build_fn=baseline_model), parameters, scoring = 'r2')
reg.fit(X_train, y_train)

#scoring against test set to see how good our model is
print(reg.score(X_test, y_test))

Epoch 1/800
 - 5s - loss: 1.2340
Epoch 2/800
 - 5s - loss: 0.3034
Epoch 3/800
 - 6s - loss: 0.2057
Epoch 4/800
 - 5s - loss: 0.1907
Epoch 5/800
 - 5s - loss: 0.0762
Epoch 6/800
 - 5s - loss: 0.0458
Epoch 7/800
 - 5s - loss: 0.0333
Epoch 8/800
 - 5s - loss: 0.0355
Epoch 9/800
 - 5s - loss: 0.0308
Epoch 10/800
 - 5s - loss: 0.0518
Epoch 11/800
 - 5s - loss: 0.1212
Epoch 12/800
 - 6s - loss: 0.1173
Epoch 13/800
 - 5s - loss: 0.1090
Epoch 14/800
 - 5s - loss: 0.1039
Epoch 15/800
 - 5s - loss: 0.1012
Epoch 16/800
 - 5s - loss: 0.0973
Epoch 17/800
 - 5s - loss: 0.0973
Epoch 18/800
 - 5s - loss: 0.0949
Epoch 19/800
 - 5s - loss: 0.0945
Epoch 20/800
 - 5s - loss: 0.0938
Epoch 21/800
 - 5s - loss: 0.0932
Epoch 22/800
 - 5s - loss: 0.0918
Epoch 23/800
 - 5s - loss: 0.0917
Epoch 24/800
 - 5s - loss: 0.0915
Epoch 25/800
 - 5s - loss: 0.0908
Epoch 26/800
 - 5s - loss: 0.0899
Epoch 27/800
 - 5s - loss: 0.0902
Epoch 28/800
 - 5s - loss: 0.0910
Epoch 29/800
 - 5s - loss: 0.0892
Epoch 30/800
 - 5s - lo

## Let's print some results and see the performance!

Code here is from Layton too.

In [10]:
#### print('data:')
print(X_test[:10])
print('\npredictions:')
print(reg.predict(X_test)[:10])
print('\ntargets:')
print(y_test[:10])

[[  3.17400000e+03  -2.70000000e-02   5.00000000e-01  -8.86000000e-01]
 [  3.30700000e+03  -4.14000000e-01   5.66400000e-01  -1.01500000e+00]
 [  3.49400000e+03   1.52300000e-01   5.62500000e-01  -8.24000000e-01]
 [  6.05300000e+03  -1.05000000e-01   5.46800000e-01  -8.43000000e-01]
 [  3.06200000e+03  -3.90000000e-02  -5.50000000e-01   7.53900000e-01]
 [  3.06700000e+03   8.51500000e-01  -3.28000000e-01   3.94500000e-01]
 [  7.71700000e+03   3.35900000e-01  -2.65000000e-01   8.35900000e-01]
 [  3.79200000e+03  -1.01000000e-01   4.49200000e-01  -8.75000000e-01]
 [  3.05900000e+03   5.50700000e-01  -4.37000000e-01   6.48400000e-01]
 [  5.63600000e+03   7.80000000e-03   6.83500000e-01  -8.39000000e-01]]

predictions:
[[-0.0421204   0.48667085 -0.89582556]
 [-0.43220752  0.54411614 -1.01299095]
 [ 0.14401543  0.51679653 -0.82244402]
 [-0.13529843  0.54258871 -0.84435093]
 [-0.06720072 -0.54357708  0.75819898]
 [ 0.81654978 -0.29333016  0.32213908]
 [ 0.2969341  -0.25199488  0.84282649]
 [